# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree()
            

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.116636e+02     1.474114e+00
 * time: 1.0173180103302002
     1     1.032957e+01     7.526683e-01
 * time: 2.10024094581604
     2    -1.131811e+01     9.858833e-01
 * time: 2.5948359966278076
     3    -3.361851e+01     7.175647e-01
 * time: 3.3524959087371826
     4    -4.666896e+01     5.255487e-01
 * time: 4.071182012557983
     5    -5.652327e+01     2.243927e-01
 * time: 4.789454936981201
     6    -5.939417e+01     2.938246e-01
 * time: 5.295464992523193
     7    -6.062438e+01     2.256006e-01
 * time: 5.782552003860474
     8    -6.120857e+01     1.438664e-01
 * time: 6.268694877624512
     9    -6.156731e+01     3.621973e-02
 * time: 6.762512922286987
    10    -6.180733e+01     9.353138e-02
 * time: 7.247684955596924
    11    -6.197040e+01     2.835554e-02
 * time: 7.733253002166748
    12    -6.207625e+01     1.694776e-02
 * time: 8.227828025817871
    13    -6.212006e+01     1.817898e-02
 * time: 8.714221954345703
    1

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671073
    AtomicLocal         -18.8557743
    AtomicNonlocal      14.8522696
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485385 
    Xc                  -19.3336825

    total               -62.261666461641